## Using Public Data from Mongol Bank Statistics and National Statistics Office of Mongolia I combined 2024's data sources. Since all the files are in excel to facilitate further usage of the data I cleaned and merged them to csv file.

In [1]:
import pandas as pd

df1 = pd.read_csv("/content/Number_of_companies.csv")  # e.g., soum-level population
df2 = pd.read_excel("/content/Population_age_districts.xlsx", skiprows = 3)
df3 = pd.read_excel("/content/Mongolian banks' loans 2019-2024.xlsx", skiprows = 5, sheet_name = "2024")
df4 = pd.read_excel("/content/Mongolian bank's current accounts and balances 2019-2024.xlsx", skiprows = 5, sheet_name = "2024")
df5 = pd.read_excel("/content/Mongolian banks' deposits 2015-2024.xlsx", skiprows = 5, sheet_name = "2024")
df6 = pd.read_excel("/content/Mongolian bank branches and personnel 2019-2024.xlsx", sheet_name = "2024", skiprows = 6)
df7 = pd.read_excel("/content/Golomt_branches.xlsx")

df3 = df3.rename(columns={
    'Үүнээс:': 'Normal',
    'Unnamed: 5': 'Delinquent',
    'Unnamed: 6': 'Overdue'
})

df5 = df5.rename(columns={
    'From which:': 'Timeless_deposits',
    'Unnamed: 5': 'Time_deposits',
    #'Unnamed: 6': 'Overdue'
})

df6 = df6.rename(columns={
    'Unnamed: 4': 'Branch',
    'Unnamed: 5': 'Settlement_center'
})

df6['Total_banks_branches'] = df6['Branch'] + df6['Settlement_center']

print("df1 columns:", df1.columns.tolist())
print("df2 columns:", df2.columns.tolist())
print("df3 columns:", df3.columns.tolist())
print("df4 columns:", df4.columns.tolist())
print("df5 columns:", df5.columns.tolist())
print("df6 columns:", df6.columns.tolist())
print("df7 columns:", df7.columns.tolist())

cols2 = ['City', 'District', 'Grand Total', 'Total_kids', 'Total_working', 'Total_olders', 'Percentage_kids', 'Percentage_working', 'Percentage_olders']
cols3 = ['City', 'District', 'Total_loan_balance','Total_loans_accounts', 'Normal', 'Delinquent', 'Overdue']
cols5 = ['City', 'District', 'Total_deposits', 'Total_deposits_accounts', 'Time_deposits', 'Timeless_deposits']
cols6 = ['City', 'District', 'Total_banks_branches']


df1 = df1.dropna(subset=['City'])
df2 = df2.dropna(subset=['City'])
df2 = df2[cols2].copy()
df2 = df2.rename(columns={'Grand Total': 'Population'})
df3 = df3[cols3].copy()
df5 = df5[cols5].copy()
df6 = df6[cols6].copy()

for df in [df1, df2, df3, df4, df5, df6, df7]:
    df.loc[:, 'City'] = df['City'].astype(str).str.strip().str.lower()
    df.loc[:, 'District'] = df['District'].astype(str).str.strip().str.lower()

branch_counts = df7.groupby(['City', 'District']).size().reset_index(name='Golomt_branches')

merged_df = pd.merge(df1, df2, on=['City', 'District'], how='inner')
merged_df = pd.merge(merged_df, df3, on=['City', 'District'], how='inner')
merged_df = pd.merge(merged_df, df4, on=['City', 'District'], how='inner')
merged_df = pd.merge(merged_df, df5, on=['City', 'District'], how='inner')
merged_df = pd.merge(merged_df, df6, on=['City', 'District'], how='inner')
merged_df = pd.merge(merged_df, branch_counts, on=['City', 'District'], how='left')
merged_df['Golomt_branches'] = merged_df['Golomt_branches'].fillna(0).astype(int)

merged_df = merged_df.drop('Number', axis=1)

print("Merged df:", len(merged_df))


merged_df

df1 columns: ['City', 'District', 'Companies']
df2 columns: ['City', 'District', '0-4', '15-19', '20-24', '25-29', '30-34', '35-39', '40-44', '45-49', '50-54', '55-59', '60-64', '65-69', '70+', '5-9', '10-14', 'Grand Total', 'Total_kids', 'Total_working', 'Total_olders', 'Percentage_kids', 'Percentage_working', 'Percentage_olders']
df3 columns: ['Number', 'City', 'District', 'Total_loan_balance', 'Normal', 'Delinquent', 'Overdue', 'Total_loans_accounts', 'Үүнээс:.1', 'Unnamed: 9', 'Unnamed: 10']
df4 columns: ['Number', 'City', 'District', 'Total_balances', 'Total_current_accounts']
df5 columns: ['Number', 'City', 'District', 'Total_deposits', 'Timeless_deposits', 'Time_deposits', 'Total_deposits_accounts', 'From which:.1', 'Unnamed: 8']
df6 columns: ['Number', 'City', 'District', 'Branches', 'Branch', 'Settlement_center', 'Unnamed: 6', 'Workers', 'Total_banks_branches']
df7 columns: ['Name', 'City', 'District', 'lat', 'lon']
Merged df: 308


,City,District,Companies,Population,Total_kids,Total_working,Total_olders,Percentage_kids,Percentage_working,Percentage_olders,...,Delinquent,Overdue,Total_balances,Total_current_accounts,Total_deposits,Total_deposits_accounts,Time_deposits,Timeless_deposits,Total_banks_branches,Golomt_branches
0,говь-алтай,алтай,21,1909,568.0,1253.0,88.0,0.297538,0.656365,0.046097,...,158.248099,207.518385,5375.042866,11871,15676.125117,5213.0,12871.332484,2804.792633,2,0
1,говь-алтай,баян-уул,34,3026,850.0,2027.0,149.0,0.280899,0.669861,0.049240,...,41.684376,8.784859,2084.376542,4858,6991.142922,2142.0,5789.870931,1201.271991,2,0
2,говь-алтай,бигэр,44,2204,616.0,1464.0,124.0,0.279492,0.664247,0.056261,...,21.461958,6.477661,2158.305947,3701,5115.532846,1490.0,4351.552852,763.979993,2,0
3,говь-алтай,бугат,22,2139,621.0,1410.0,108.0,0.290323,0.659187,0.050491,...,42.508236,30.616131,1630.978377,3916,7613.467698,1532.0,6342.170473,1271.297225,2,0
4,говь-алтай,дэлгэр,49,3032,802.0,2069.0,161.0,0.264512,0.682388,0.053100,...,126.604846,124.117105,1950.544576,5504,5865.064568,1949.0,4751.227161,1113.837407,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303,хэнтий,батноров,73,5731,1687.0,3678.0,366.0,0.294364,0.641773,0.063863,...,21.593898,36.717167,1874.144549,5170,5572.657733,1653.0,4432.277263,1140.38047,2,0
304,хэнтий,баян-овоо,26,1934,601.0,1228.0,105.0,0.310755,0.634953,0.054292,...,32.783742,40.236375,1060.939288,3103,2603.308827,1317.0,2023.645451,579.663376,2,0
305,хэнтий,мөрөн,32,1845,459.0,1264.0,122.0,0.248780,0.685095,0.066125,...,58.793751,32.894776,899.805254,2717,1179.384067,728.0,847.258737,332.12533,2,0
306,хэнтий,хэрлэн,570,24764,8202.0,15051.0,1511.0,0.331207,0.607777,0.061016,...,1023.566577,959.803574,16899.453091,34770,55404.753895,16685.0,43767.090607,11637.663288,9,1


# Checking dtypes

In [2]:
merged_df.dtypes

,0
City,object
District,object
Companies,object
Population,int64
Total_kids,float64
Total_working,float64
Total_olders,float64
Percentage_kids,float64
Percentage_working,float64
Percentage_olders,float64


# Since all dtypes except city and district should be numbers, for columns that dont match these types I converted to float.

(The dtype error caused due to usage of comma within the numbers)

In [3]:
col = ['Companies', 'Normal', 'Delinquent', 'Overdue', 'Time_deposits', 'Timeless_deposits', 'Total_banks_branches']

for c in col:
    merged_df[c] = merged_df[c].astype(str).str.replace(',', '')
    merged_df[c] = pd.to_numeric(merged_df[c], errors='coerce')
    merged_df[c] = merged_df[c].astype('float64')

#To make it user friendly to run in another code sections I made it to available to download the merged data

In [ ]:
from google.colab import files
merged_df.to_excel('merged_df.xlsx', index=False)
files.download('merged_df.xlsx')
